# Applied SQL Analytics: Chapter 2

---
---

## SQL Joins

### Inner Join

Only the records that exist in both tables are returned.

Limiting the output to only one of the joined tables with `table.*`:

```SQL
SELECT salespeople.*
FROM salespeople
INNER JOIN dealerships
  ON dealerships.dealership_id = salespeople.dealership_id
WHERE dealerships.state = 'CA'
ORDER BY 1;
```

Aliasing table names, with our without `AS`:

```SQL
SELECT s.*
FROM salespeople AS s
INNER JOIN dealerships d
  ON d.dealership_id = s.dealership_id
WHERE d.state = 'CA'
ORDER BY 1;
```

### Outer Join

All records from the table in question are returned, and all matching from the other.

```SQL
SELECT *
FROM emails e
RIGHT OUTER JOIN customers c ON e.customer_id=c.customer_id
ORDER BY c.customer_id
LIMIT 1000;
```

### Cross Join

Returns every possible combination of records from the two tables.

Not generally done in practice, but good to know it exists.

#### Using Joins

The head of sales at your company would like a list of all customers who bought a car. We need to create a query that will return all customer IDs, first names, last names, and valid phone numbers of customers who purchased a car.

```SQL
SELECT
	c.customer_id,
	c.first_name,
	c.last_name,
	c.phone
FROM sales s
INNER JOIN customers c ON c.customer_id = s.customer_id
INNER JOIN products p ON p.product_id = s.product_id
WHERE p.product_type = 'automobile'
AND c.phone IS NOT NULL;
```

---

## Subqueries

Subqueries are a way to use the tables produced by the SELECT queries instead of referencing an existing table in your database. You can simply take a query, insert it between a pair of parentheses, and give it an alias.

Find all the salespeople working in California:

```SQL
SELECT *
FROM salespeople
INNER JOIN (
  SELECT * FROM dealerships
  WHERE dealerships.state = 'CA'
  ) d
  ON d.dealership_id = salespeople.dealership_id
ORDER BY 1;
```

If a query only has one column, you can use a subquery with the IN keyword in a WHERE clause. To extract the details from the salespeople table using the dealership ID for the state of California:

```SQL
SELECT *
FROM salespeople
WHERE dealership_id IN (
  SELECT dealership_id from dealerships
  WHERE dealerships.state = 'CA'
  )
ORDER BY 1;
```

---

## Unions

With joins, columns are added "horizontally" - _columns_ are added. Unions can keep the same number of columns but add together the rows of multiple queries.

To visualize the addresses of dealerships and customers using Google Maps, you would need both the addresses of customers and dealerships.

```SQL
(
  SELECT street_address, city, state, postal_code
  FROM customers
  WHERE street_address IS NOT NULL
)
UNION
(
  SELECT street_address, city, state, postal_code
  FROM dealerships
  WHERE street_address IS NOT NULL
)
ORDER BY 1;
```

Notes:

* The subqueries result in columns with the same names and data types
* `UNION` by default removes all duplicate rows in the output
  * `UNION ALL` retains the duplicate rows

##### Exercise 2.02

Make a guest list with ZoomZoom customers who live in Los Angeles, CA, as well as salespeople who work at the ZoomZoom dealership in Los Angeles, CA. The guest list should include first and last names and whether the guest is a customer or an employee.

```SQL
(
  SELECT first_name, last_name, 'Customer' as guest_type
  FROM customers
  WHERE city = 'Los Angeles'
  AND state = 'CA'
)
UNION
(
  SELECT first_name, last_name, 'Employee' as guest_type
  FROM salespeople s
  INNER JOIN dealerships d on d.dealership_id = s.dealership_id
  WHERE d.city = 'Los Angeles'
  AND d.state = 'CA'
)
```

---

## Common Table Expressions

* Similar to subqueries
* Create temporary tables using the `WITH` clause
* One advantage is that common table expressions are recursive
  * Recursive common table expressions can reference themselves
  
```SQL
WITH d as (
SELECT * FROM dealerships
  WHERE dealerships.state = 'CA'
  )
SELECT *
FROM salespeople
INNER JOIN d ON d.dealership_id = salespeople.dealership_id
ORDER BY 1;
```

---

## CASE WHEN

> Transforming Data

Process the query output data, like removing or subsituting values, or mapping values to other values.

##### The `CASE WHEN` Function

`CASE WHEN` is a function to map values in a column to other values. Here's the general form:

```SQL
CASE WHEN condition1 THEN value1
WHEN condition2 THEN value2
…
WHEN conditionX THEN valueX
ELSE else_value END;
```

Example:

* Return all rows for customers from the customers table
* Add a column `customer_type` that labels a user as:
  * Elite Customer type if they live in postal code 33111
  * Premium Customer type if they live in postal code 33124
  * Otherwise, it will mark the customer as a Standard Customer type.

```SQL
SELECT *,
  CASE WHEN postal_code='33111' THEN 'Elite Customer'
  WHEN postal_code='33124' THEN 'Premium Customer'
  ELSE 'Standard Customer' END
 AS customer_type
FROM customers;
```

Exercise 2.03:

* Customers from the states of MA, NH, VT, ME, CT, or RI
  * Label as New England
* Customers from the states of GA, FL, MS, AL, LA, KY, VA, NC, SC, TN, VI, WV, or AR
  * Labeled as Southeast
* Customers from any other state should be labeled as Other

```SQL
SELECT c.customer_id,
CASE WHEN c.state in ('MA', 'NH', 'VT', 'ME', 'CT', 'RI') THEN 'New England'
WHEN c.state in ('GA', 'FL', 'MS', 'AL', 'LA', 'KY', 'VA', 'NC', 'SC', 'TN', 'VI', 'WV', 'AR') THEN 'Southeast'
ELSE 'Other' END as region
FROM customers c
ORDER BY 1;
```

---

## The COALESCE Function

Replace `NULL` values with a standard value. Goes through provided list of columns and scalar values, filling any nulls in the former with the latter.

Get a list of the first names, last names, and phone numbers of all male customers. For those customers with no phone number, instead write the value of 'NO PHONE'.

```sql
SELECT first_name,
last_name,
COALESCE(phone, 'NO PHONE') as phone
FROM customers
ORDER BY 1;
```

---

## The NULLIF Function

Two-value function (like COALESCE) that returns `NULL` if the first value equals the second value.

This example removes the title "Honorable" from those that have it.

```sql
SELECT customer_id,
  NULLIF(title, 'Honorable') as title,
  first_name,
  last_name,
  suffix,
  email,
  gender,
  ip_address,
  phone,
  street_address,
  city,
  state,
  postal_code,
  latitude,
  longitude,
  date_added
FROM customers c
ORDER BY 1;
```

---

## The LEAST/GREATEST Functions

Takes any number of values and returns the least/greatest of the values. For example, replacing a value if it's too high or low.

```sql
SELECT product_id,
  model,
  year,
  product_type,
  LEAST(600.00, base_msrp) as base_msrp,
  production_start_date,
  production_end_date
FROM products
WHERE product_type='scooter'
ORDER BY 1;
```

---

## The Casting Function

Change the data type of a column within a query using the `column::datatype` syntax.

```sql
SELECT product_id,
  model,
  year::TEXT,
  product_type,
  base_msrp,
  production_start_date,
  production_end_date
FROM products;
```

---

## The DISTINCT and DISTINCT ON Functions

I already have a pretty good grasp of distinct, though I don't think I knew that it can be used on multiple columns to get all of the distinct column combinations.

```sql
SELECT DISTINCT year, product_type
FROM products
ORDER BY 1, 2;
```

The related keyword `DISTINCT ON` ensures that only one row is returned and one or more columns are always unique in the set.

This example selects salespeople with unique first names, and if there are two identical first names, the one who was hired first will be returned.

```sql
SELECT DISTINCT ON (first_name)
*
FROM salespeople
ORDER BY first_name, hire_date;
```

---
---

## Activity 2.02: Building a Sales Model Using SQL Techniques

In this activity, we will clean and prepare our data for analysis using SQL techniques. The data science team wants to build a new model to help predict which customers are the best prospects for remarketing. A new data scientist has joined their team and does not know the database well enough to pull a dataset for this new model. The responsibility has fallen to you to help the new data scientist prepare and build a dataset to be used to train a model. Write a query to assemble a dataset. Here are the steps to perform:

1. Open a SQL client and connect to the database.
2. Use `INNER JOIN` to join the `customers` table to the `sales` table.
3. Use `INNER JOIN` to join the `products` table to the `sales` table.
4. Use `LEFT JOIN` to join the `dealerships` table to the `sales` table.
5. Now, return all columns of the `customers` table and the `products` table.
6. Then, return the `dealership_id` column from the `sales` table, but fill in `dealership_id` in `sales` with `-1` if it is `NULL`.
7. Add a column called `high_savings` that returns `1` if the sales amount was `500` less than `base_msrp` or lower. Otherwise, it returns `0`.

2. `sales` and `customers` can be joined on the `customer_id` column

```SQL
SELECT *
FROM sales s
INNER JOIN customers c
  ON c.customer_id = s.customer_id;
```

3. `sales` and `products` can be joined on `product_id`

```SQL
SELECT *
FROM sales s
INNER JOIN products p
  ON p.product_id = s.product_id;
```

4. `sales` and `dealerships` can be joined on `dealership_id`

```SQL
SELECT *
FROM sales s
LEFT JOIN dealerships d
  ON d.dealership_id = s.dealership_id;
```

Bringing all of those together (plus the proceeding steps):

```SQL
SELECT
  c.*,
  p.*,
  COALESCE(s.dealership_id, -1),
  CASE WHEN s.sales_amount < p.base_msrp - 500 THEN 1
  ELSE 0 END as high_savings
FROM sales s
INNER JOIN customers c ON c.customer_id = s.customer_id
INNER JOIN products p ON p.product_id = s.product_id
LEFT JOIN dealerships d ON d.dealership_id = s.dealership_id;
```